<a href="https://colab.research.google.com/github/revati2013/Assignment05_Multi-Linear-Regression/blob/main/Assignment_5(50_startup).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import statsmodels.formula.api as smf
import statsmodels.api as sm
from statsmodels.graphics.regressionplots import influence_plot

In [ ]:
d=pd.read_csv('50_Startups.csv')

In [ ]:
d

In [ ]:
data=d.rename({'R&D Spend':'RDS','Administration':'Adms','Marketing Spend':'MS'},axis=1)

In [ ]:
data.info()

In [ ]:
data[data.duplicated()]

No duplicate data

In [ ]:
data.describe()

In [ ]:
#correlation analysis
data.corr()

In [ ]:
sns.pairplot(data)

In [ ]:
#model building
model=smf.ols("Profit~RDS+Adms+MS",data=data).fit()

In [ ]:
model.params

In [ ]:
#finding tvalues and pvalues
model.tvalues,np.round(model.pvalues,5)

In [ ]:
#finding rsquared values
model.rsquared , model.rsquared_adj 

Model accuracy is 94.75%

In [ ]:
# Building SLR and MLR models for insignificant variables 'ADMS' and 'MKTS'
# Also finding their tvalues and pvalues
slr_a=smf.ols("Profit~Adms", data=data).fit()
slr_a.tvalues, slr_a.pvalues

In [ ]:
slr_m=smf.ols("Profit~MS",data=data).fit()
slr_m.tvalues, slr_m.pvalues

In [ ]:
mlr_am=smf.ols("Profit~Adms+MS",data=data).fit()
mlr_am.tvalues, mlr_am.pvalues

In [ ]:
# 1) Collinearity Problem Check
# Calculate VIF = 1/(1-Rsquare) for all independent variables

rsq_r=smf.ols("RDS~Adms+MS",data=data).fit().rsquared
vif_r=1/(1-rsq_r)

rsq_a=smf.ols("Adms~RDS+MS",data=data).fit().rsquared
vif_a=1/(1-rsq_a)

rsq_m=smf.ols("MS~RDS+Adms",data=data).fit().rsquared
vif_m=1/(1-rsq_m)

# Putting the values in Dataframe format
d1={'Variables':['RDS','Adms','MS'],'Vif':[vif_r,vif_a,vif_m]}
Vif_df=pd.DataFrame(d1)
Vif_df

None variable has VIF>20,no collinearity

In [ ]:
# 2) Residual Analysis
# Test for Normality of Residuals (Q-Q Plot) using residual model (model.resid)
sm.qqplot(model.resid,line='q')
plt.title("Normal QQ plot of residual")
plt.show()

In [ ]:
# Test for Homoscedasticity or Heteroscedasticity (plotting model's standardized fitted values vs standardized residual values)

def standard_values(vals) : return (vals-vals.mean())/vals.std()  # User defined z = (x - mu)/sigma

plt.scatter(standard_values(model.fittedvalues),standard_values(model.resid))
plt.title('Residual Plot')
plt.xlabel('standardized fitted values')
plt.ylabel('standardized residual values')
plt.show() 

In [ ]:
# Test for errors or Residuals Vs Regressors or independent 'x' variables or predictors 
fig=plt.figure(figsize=(15,8))
sm.graphics.plot_regress_exog(model,'RDS',fig=fig)
plt.show()

In [ ]:
fig=plt.figure(figsize=(15,8))
sm.graphics.plot_regress_exog(model,'Adms',fig=fig)
plt.show()

In [ ]:
fig=plt.figure(figsize=(15,8))
sm.graphics.plot_regress_exog(model,'MS',fig=fig)
plt.show()

In [ ]:
#Model Deletion Diagnostics (checking Outliers or Influencers)
# Two Techniques : 1. Cook's Distance & 2. Leverage value
(c,_)=model.get_influence().cooks_distance

In [ ]:
c

In [ ]:
# Plot the influencers using the stem plot
fig=plt.figure(figsize=(15,7))
plt.stem(np.arange(len(data)),np.round(c,5))
plt.xlabel('Row Index')
plt.ylabel('Cooks Distance')
plt.show()

In [ ]:
# Index and value of influencer where C>0.5
np.argmax(c),np.max(c)

In [ ]:
# 2. Leverage Value using High Influence Points
influence_plot(model)
plt.show()

In [ ]:
data[data.index.isin([49])]

In [ ]:
data1=data.drop(data.index[[49]],axis=0)

In [ ]:
data1

In [ ]:
 # Final Model
 final_model=smf.ols("Profit~RDS+Adms+MS",data=data1).fit()
 final_model.rsquared , final_model.aic

model accuracy is improved to 96.13%

In [ ]:
#New data for prediction is
new_data=pd.DataFrame({'RDS':70000,"Adms":90000,"MS":140000},index=[0])
new_data

In [ ]:
final_model.predict(new_data)

In [ ]:
# Automatic Prediction of Price with 90.02% accurcy
pred_y=final_model.predict(data1)
pred_y

In [ ]:
#table containing R^2 value for each prepared model
d2={'Prep_Models':['Model','Final_Model'],'Rsquared':[model.rsquared,final_model.rsquared]}
table=pd.DataFrame(d2)

In [ ]:
table